# W207, Final Project
Spring, 2018

Team:  Cameron Kennedy, Gaurav Khanna, Aaron Olson

## Data Preparation / Feature Extraction Notebook
Python Notebook 1 of 2

This notebook loads and pre-processes the data.  The other notebook (2 of 2) runs our ML models.

# Introduction
This analysis seeks to predict user churn in a music sharing service.

We will write a more complete description and analysis for submission of our final project.

In [1]:
#Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Load the data
members = pd.read_csv('members_filtered.csv')
transactions = pd.read_csv('transactions_filtered.csv')
user_logs = pd.read_csv('user_logs_filtered.csv')
labels = pd.read_csv('labels_filtered.csv')

#Set indices
members.set_index('msno', inplace = True)
labels.set_index('msno', inplace = True)

#user_logs.head()

In [3]:
def pd_to_date(df_col):
    df_col = pd.to_datetime(df_col, format = '%Y%m%d')
    return df_col

#Convert to date
user_logs['date'] = pd_to_date(user_logs['date'])
#user_logs.head()

# User Logs Data: Preparation and Feature Extraction

In [4]:
#Create our groupby user object 
user_logs_gb = user_logs.groupby(['msno'], sort=False)

In [5]:
#This cell is slow

#Append max date to every row in main table
user_logs['max_date'] = user_logs_gb['date'].transform('max')
user_logs['days_before_max_date'] = (user_logs['max_date'] - user_logs['date']).apply(lambda x: x.days)
    #The .apply(lambda...  just converts it from datetime to an integer, for easier comparisons later.

#Generate user's first date, last date, and tenure
#Also, the user_logs_features table will be the primary table to return from the transactions table
user_logs_features = (user_logs_gb
    .agg({'date':['max', 'min', lambda x: (max(x) - min(x)).days]})  #.days converts to int
    .rename(columns={'max': 'max_date', 'min': 'min_date','<lambda>':'listening_tenure'})
                      )
#Add a 3rd level, used for joining data later
user_logs_features = pd.concat([user_logs_features], axis=1, keys=['date_features'])

In [6]:
user_logs_features.head()

date_features             \
                                                      date              
                                                  max_date   min_date   
msno                                                                    
AtRE9/xHjNrX8tcuUb/cdBRvUHpwXhZrWY9E9sk1tDk=    2017-02-24 2016-01-25   
f3HbYjEEVBVeWcP3FNDCLKJXpuIZPvX0oFyn4XLMN3k=    2017-02-28 2015-01-02   
Q7nR6DbodHwUTGPi5Z0yOsdC2cSOhiWIe4cNEVmnJ9I=    2017-02-28 2016-02-07   
Qx9cihonlt2hkBa3VHNj+nwX2QelZXpyHnEAIjD199w=    2017-02-24 2015-01-02   
mf3zFfyeZLl8zEWhV+GyWQYjt3orfQUCcOIMwBJ60+k=    2017-02-28 2015-12-20   

                                                               
                                                               
                                             listening_tenure  
msno                                                           
AtRE9/xHjNrX8tcuUb/cdBRvUHpwXhZrWY9E9sk1tDk=              396  
f3HbYjEEVBVeWcP3FNDCLKJXpuIZPvX0oFyn4XLMN3k=              788  
Q7nR6DbodHwUTGPi5Z0yOsdC2cSOhiWIe4cNEVmnJ9I=              387  
Qx9cihonlt2hkBa3VHNj+nwX2QelZXpyHnEAIjD199w=              784  
mf3zFfyeZLl8zEWhV+GyWQYjt3orfQUCcOIMwBJ60+k=              436

In [7]:
#Create Features:
    # Total X=(seconds, 100, 985, 75, 50, 25, unique), avg per day of X, maybe median per day of X
    # Last day, last 7 days, last 30 days, last 90, 180, 365, total (note last day is relative to user)
    
for num_days in [7, 14, 31, 90, 180, 365, 999]:
    #Create groupby object for items with x days
    ul_gb_xdays = (user_logs.loc[(user_logs['days_before_max_date'] < num_days)]
                   .groupby(['msno'], sort=False))

    #Generate sum and mean (and count, once) for all the user logs stats
    past_xdays_by_user = (ul_gb_xdays
        .agg({'num_unq':['sum', 'mean', 'count'],
              'total_secs':['sum', 'mean'],
              'num_25':['sum', 'mean'],
              'num_50':['sum', 'mean'],
              'num_75':['sum', 'mean'],
              'num_985':['sum', 'mean'],
              'num_100':['sum', 'mean'],
             })
                      )
    #Append level header
    past_xdays_by_user = pd.concat([past_xdays_by_user], axis=1, keys=['within_days_' + str(num_days)])

    #Join (append) to user_logs_features table
    user_logs_features = user_logs_features.join(past_xdays_by_user, how='inner')

In [8]:
#Next, let's look at changes in last 7 days vs. last 30 days, and last 30 days vs. last 180 days.

#Also, need to think about users with < x days tenure.

In [9]:
#Join members and labels files
features_all = None
features_all = members.join(labels, how='inner')
features_all = features_all.join(user_logs_features, how='inner')

#Note, the warning is okay, and actually helps us by flattening our column headers.

C:\Users\camke\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:551: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 3 on the right)
  warnings.warn(msg, UserWarning)


# Transaction Data: Preparation and Feature Extraction

# Final Join of Transactions into Features_All Table

In [ ]:
features_all = features_all.join(<GAURAV INSERT YOUR DATAFRAME HERE>, how='inner')

In [10]:
#Write all features to pkl
features_all.to_pickle('features_all.pkl')